# memory

- 순수 openai api로는 chat gpt와 대화를 기억하게 할 수 없지만, 랭체인은 가능하다.
- 모델이 모든 대화를 기억할 수 없기에, 이 전의 대화도 함께 모델서버에 전송하게 된다.

### conversation buffer memory

- 새로운 프롬프트를 입력할 때마다, 이전에 기록한 프롬프트를 함께 서버에 전송한다. 프롬프트가 너무 길어져서 비용이 많이 나온다.

In [5]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True) # 리턴 메세지를 트루로 해야 대답이 온다.

memory.save_context({"input": "hi!"}, {"output": "how are you?"})

memory.load_memory_variables({}) # 이상하지만 히스토리가 보인다

{'history': [HumanMessage(content='hi!'), AIMessage(content='how are you?')]}

- 반복할수록 대화가 전부 기록 된다.

In [6]:
memory.save_context({"input": "hi!"}, {"output": "how are you?"})

memory.load_memory_variables({}) # 이상하지만 히스토리가 보인다

{'history': [HumanMessage(content='hi!'),
  AIMessage(content='how are you?'),
  HumanMessage(content='hi!'),
  AIMessage(content='how are you?')]}

### conversation buffer window memory

- 새로운 프롬프트와 함께 이전에 입력한 프롬프트들 중에서 일정한 갯수만 서버에 전송한다. 예를 들면, 최근 5개 메세지만 저장하고 서버에 전송하게 할 수 있다. 가장 오래된 프롬프트는 삭제된다. 큐와 원리가 비슷하다.

In [13]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output):
    memory.save_context({"input": input},{"output": output})

add_message(1,1)

In [14]:
add_message(2,2)
add_message(3,3)
add_message(4,4)

In [15]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [16]:
add_message(5,5)

In [18]:
# 윈도우가 움직이면서 최근 5개 메세지만 보여준다.

memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

## conversation summary memory
- 이전에 저장한 프롬프트의 갯수가 많아서 비용이 많이 나온다면, 이전 프롬프트들을 요약해서 서버에 전송한다.

In [1]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

# human, ai messages
add_message("Hi I'm Lake, I live in South Korea,", "Wow that is so cool!")

In [2]:
add_message("South Korea is so pretty", "I wish I could go!")

In [3]:
get_history()

{'history': 'The human introduces themselves as Lake and mentions that they live in South Korea. The AI responds by expressing admiration for this information and expresses a desire to visit South Korea because it is so pretty.'}

### conversation summary buffer memory

- conversation buffer와 summary를 결합한 라이브러리다.
- 최근 프롬프트들을 저장해서 입력한 프롬프트 자체로 사용하고, 일정한 갯수가 넘어가면 가장 오래된 프롬프트들을 요약한다.

In [7]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=150,
    return_messages=True
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

In [8]:
add_message("Hi I'm Lake, I live in South Korea,", "Wow that is so cool!")

In [9]:
get_history()

{'history': [HumanMessage(content="Hi I'm Lake, I live in South Korea,"),
  AIMessage(content='Wow that is so cool!')]}

In [10]:
add_message("South Korea is so pretty", "I wish I could go!")

In [11]:
get_history()

{'history': [HumanMessage(content="Hi I'm Lake, I live in South Korea,"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!')]}

In [12]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")

In [13]:
get_history()

{'history': [HumanMessage(content="Hi I'm Lake, I live in South Korea,"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}

In [14]:
add_message("How far is Korea from Brazil?", "I don't know! Super far!")

In [16]:
get_history()

{'history': [HumanMessage(content="Hi I'm Lake, I live in South Korea,"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Korea from Brazil?'),
  AIMessage(content="I don't know! Super far!")]}

In [17]:
add_message("How far is Korea from USA?", "I don't know! Super far!")

In [19]:
get_history()

{'history': [HumanMessage(content="Hi I'm Lake, I live in South Korea,"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Korea from Brazil?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Korea from USA?'),
  AIMessage(content="I don't know! Super far!")]}

In [20]:
add_message("How far is Korea from Canada?", "I don't know! Super far!")

In [21]:
get_history()

{'history': [SystemMessage(content='The human introduces themselves as Lake and mentions that they live in South Korea.'),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Korea from Brazil?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Korea from USA?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Korea from Canada?'),
  AIMessage(content="I don't know! Super far!")]}

### conversation kg memory
- kg는 knowledge
- 대화 엔티티의 knowledge graph를 만든다.
- 가장 중요한 히스토리를 요약한다.

In [22]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message("Hi I'm Lake, I live in South Korea", "Wow that is so cool")

In [23]:
memory.load_memory_variables({"input": "who is Lake?"})

{'history': [SystemMessage(content='On Lake: Lake is human. Lake lives in South Korea.')]}

In [24]:
add_message("Lake likes to eat hamburger", "Wow that is so cool")

In [25]:
memory.load_memory_variables({"input": "What does Lake like?"})

{'history': [SystemMessage(content='On Lake: Lake is human. Lake lives in South Korea. Lake likes to eat hamburger.')]}

### memory on llmchain

- 메모리를 체인에 플러그하는 방법은 2가지가 있다.
- llm chain은 off-the-shelf chain이고, 일반적인 타스크를 목적으로 사용된다.
- 반면에 langchain expessrion을 이용하면 체인을 커스텀화할 수 있다.

### llm chain

In [38]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

template = """
    You are a helpful AI talking to a human.
    {chat_history}
    Human: {question}
    You:
"""

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    memory_key="chat_history"
)

chain = LLMChain(
    llm=llm, memory=memory, prompt=PromptTemplate.from_template(template), verbose=True,
)

chain.predict(question="My name is Lake")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human: My name is Lake
    You:


> Finished chain.


'Hello Lake! How can I assist you today?'

In [39]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    Human: My name is Lake
AI: Hello Lake! How can I assist you today?
    Human: I live in Seoul
    You:


> Finished chain.


"That's great! Seoul is a vibrant city with a rich history and culture. How can I assist you today, Lake?"

In [41]:
# 메모리는 계속 업데이트 되고 있지만, 반영이 되고 있지 않는다.
memory.load_memory_variables({})

{'chat_history': "System: The human introduces themselves as Lake.\nAI: Hello Lake! How can I assist you today?\nHuman: I live in Seoul\nAI: That's great! Seoul is a vibrant city with a rich history and culture. How can I assist you today, Lake?\nHuman: What is my name?\nAI: Your name is Lake."}

In [40]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    Human: My name is Lake
AI: Hello Lake! How can I assist you today?
Human: I live in Seoul
AI: That's great! Seoul is a vibrant city with a rich history and culture. How can I assist you today, Lake?
    Human: What is my name?
    You:


> Finished chain.


'Your name is Lake.'

### chat based memory

- 메모리는 두가지 방식으로 데이터를 출력한다. 스트링과 메세지 형태다.
- 그동안 스트링으로 메모리를 출력했다. memory.load_memory_variables({})
- 메세지로 출력할 때는 메세지플레이스홀더로 메세지를 담는 공간을 설정한다.

In [42]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    memory_key="chat_history",
    return_messages=True  # 메세지 클래스로 출력
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),

])

chain = LLMChain(
    llm=llm, memory=memory, prompt=prompt, verbose=True,
)

chain.predict(question="My name is Lake")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Lake

> Finished chain.


'Hello Lake! How can I assist you today?'

### lcel based memory

- 매번 메모리를 불러오는 방식보다는 러너블패스스루를 이용하면 더 편리하다.
- load_memory 아웃풋과 chain.invoke의 question이 결합돼서 prompt의 인자로 사용이 된다.

In [46]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    memory_key="chat_history",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),

])

def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({
        "question": question
    })
    memory.save_context({"input": question}, {"output": result.content})
    print(result)

invoke_chain("")